In [8]:
from llama_cpp import Llama
llm = Llama(model_path="/home/vinhdq/matrix/ASR_and_Summarize/asr_summarize_app/models/llama2/llama-2-13b-chat.ggmlv3.q4_K_M.bin")

llama.cpp: loading model from /home/vinhdq/matrix/ASR_and_Summarize/asr_summarize_app/models/llama2/llama-2-13b-chat.ggmlv3.q4_K_M.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 15 (mostly Q4_K - Medium)
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.09 MB
llama_model_load_internal: mem required  = 9336.95 MB (+ 1608.00 MB per state)
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16

In [9]:
content_to_sum = "This man is so fast. Once upon a time I saw him running around in flash time. One day, I found him dead."

In [10]:
adu = llm(prompt=f"### Instruction: Summarize the following, return the summarized text only: {content_to_sum}\n### Response:",stream=True)

In [11]:
for i in adu:
    print(i['choices'][0]['text'])



This
 man
 was
 incred
ibly
 fast
 and
 could
 run
 at
 an
 aston
ishing
 speed
,
 almost
 like
 he
 was
 moving
 in
 flash
 time
.
 Unfortunately
,
 one
 day
 he
 was
 found
 dead
.




llama_print_timings:        load time = 29918.42 ms
llama_print_timings:      sample time =    16.52 ms /    35 runs   (    0.47 ms per token,  2118.13 tokens per second)
llama_print_timings: prompt eval time = 29918.30 ms /    50 tokens (  598.37 ms per token,     1.67 tokens per second)
llama_print_timings:        eval time = 21212.05 ms /    34 runs   (  623.88 ms per token,     1.60 tokens per second)
llama_print_timings:       total time = 51225.34 ms


In [12]:
from whispercpp import Whisper

w = Whisper.from_pretrained("/home/vinhdq/matrix/ASR_and_Summarize/asr_summarize_app/models/whisper/ggml-tiny.en.bin")

whisper_init_from_file_no_state: loading model from '/home/vinhdq/matrix/ASR_and_Summarize/asr_summarize_app/models/whisper/ggml-tiny.en.bin'
whisper_model_load: loading model
whisper_model_load: n_vocab       = 51864
whisper_model_load: n_audio_ctx   = 1500
whisper_model_load: n_audio_state = 384
whisper_model_load: n_audio_head  = 6
whisper_model_load: n_audio_layer = 4
whisper_model_load: n_text_ctx    = 448
whisper_model_load: n_text_state  = 384
whisper_model_load: n_text_head   = 6
whisper_model_load: n_text_layer  = 4
whisper_model_load: n_mels        = 80
whisper_model_load: f16           = 1
whisper_model_load: type          = 1
whisper_model_load: mem required  =  129.00 MB (+    3.00 MB per decoder)
whisper_model_load: adding 1607 extra tokens
whisper_model_load: model ctx     =   73.58 MB
whisper_model_load: model size    =   73.54 MB
whisper_init_state: kv self size  =    2.62 MB
whisper_init_state: kv cross size =    8.79 MB


In [21]:
import ffmpeg
import numpy as np

try:
    y, _ = (
        ffmpeg.input("/home/vinhdq/matrix/ASR_and_Summarize/whisper.cpp/samples/sample1.wav", threads=0)
        .output("-", format="s16le", acodec="pcm_s16le", ac=1, ar=16000)
        .run(
            cmd=["ffmpeg", "-nostdin"], capture_stdout=True, capture_stderr=True
        )
    )
except ffmpeg.Error as e:
    raise RuntimeError(f"Failed to load audio: {e.stderr.decode()}") from e

arr = np.frombuffer(y, np.int16).flatten().astype(np.float32) / 32768.0

w.transcribe(arr)

" It wasn't like I was asking for the code to uniquely a bunker or anything like that, but the amount of resistance I got from this"

In [13]:
for i in w.stream_transcribe(5000,sample_rate=16000,step_ms=2500,n_threads=8):
    print(i)


init_device: Using device: '(null)' ...

Opened audio device: (id=2, name=(null))
  - sample_rate: 16000
  - format: 33056 (required: 33056)
  - channels: 1 (required: 1)
  - samples per frame: 1024


stream_transcribe: processing 40000 samples (step = 2.5 sec / len = 5.0 sec / keep = 0.2 sec), 8 threads, lang = en, task = transcribe, timestamps = 0 ...
stream_transcribe: n_new_line = 1, no_context = 1

=== Transcription starting now... ===



 That's your door, you can't call                                                                   
 Call me anything or anything                                                                       
 Just don't call me off                                                                             
 (upbeat music)                                                                                     
 (upbeat music)                                                                                     
 (upbeat music)                                                                                     
 (upbeat music)                                                                                     
 (upbeat music)                                                                                     
 (upbeat music)                                                                                     
 (upbeat music)                                                                            

In [14]:
dm = w.stream_transcribe(5000,sample_rate=16000,step_ms=2500,n_threads=8)


init_device: Using device: '(null)' ...

Opened audio device: (id=2, name=(null))
  - sample_rate: 16000
  - format: 33056 (required: 33056)
  - channels: 1 (required: 1)
  - samples per frame: 1024


stream_transcribe: processing 40000 samples (step = 2.5 sec / len = 5.0 sec / keep = 0.2 sec), 8 threads, lang = en, task = transcribe, timestamps = 0 ...
stream_transcribe: n_new_line = 1, no_context = 1

=== Transcription starting now... ===



 ♪ Don't tell me ♪                                                                                  
 (upbeat music)                                                                                     
 (upbeat music)                                                                                     
 (upbeat music)                                                                                     
 (upbeat music)                                                                                     
 [Music]                                                                                            
 (dramatic music)                                                                                   
 (dramatic music)                                                                                   
 (upbeat music)                                                                                     
 (gentle music)                                                                            